In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
file_path=("/content/drive/MyDrive/flight delay prediction/Airline Delay (1).csv")

In [ ]:
import pandas as pd
file_path=("/content/drive/MyDrive/flight delay prediction/Airline Delay (1).csv")
df=pd.read_csv(file_path)
df.head()

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,nas_ct,carrier_delay,weather_delay
0,2020,12,9E,Endeavor Air Inc.,ABE,"Allentown/Bethlehem/Easton, PA: Lehigh Valley ...",44.0,3.0,1.63,0.0,0.12,56.0,0.0
1,2020,12,9E,Endeavor Air Inc.,ABY,"Albany, GA: Southwest Georgia Regional",90.0,1.0,0.96,0.0,0.04,22.0,0.0
2,2020,12,9E,Endeavor Air Inc.,AEX,"Alexandria, LA: Alexandria International",88.0,8.0,5.75,0.0,1.60,265.0,0.0
3,2020,12,9E,Endeavor Air Inc.,AGS,"Augusta, GA: Augusta Regional at Bush Field",184.0,9.0,4.17,0.0,1.83,192.0,0.0
4,2020,12,9E,Endeavor Air Inc.,ALB,"Albany, NY: Albany International",76.0,11.0,4.78,0.0,5.22,0.0,NaN


DATA PREPROCESSING MODULE

In [ ]:
import pandas as pd
import numpy as np

# Corrected file path with spaces and special characters
file_path = '/content/drive/MyDrive/flight delay prediction/Airline Delay (1).csv'
df = pd.read_csv(file_path)

# Function to preprocess numeric columns
def preprocess_column(col):
    if pd.api.types.is_numeric_dtype(col):
        # Replace values that are not non-negative integers with NaN
        col = col.apply(lambda x: x if pd.notnull(x) and x >= 0 and float(x).is_integer() else np.nan)
        return col.dropna().astype(int)
    else:
        return col  # Keep string/object columns as-is

# Apply preprocessing
processed_df = df.apply(preprocess_column)

# Drop rows with any NaNs
processed_df.dropna(inplace=True)

# Save cleaned file
output_path = '/content/drive/MyDrive/flight delay prediction/Airline_Delay_Cleaned.csv'
processed_df.to_csv(output_path, index=False)

print(f"✅ Preprocessing complete. Cleaned dataset saved to: {output_path}")


✅ Preprocessing complete. Cleaned dataset saved to: /content/drive/MyDrive/flight delay prediction/Airline_Delay_Cleaned.csv


FEATURE EXTRACTION MODULE

In [72]:
import pandas as pd
import numpy as np
from datetime import timedelta, datetime
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# === Utility Functions ===

def generate_random_time():
    hour = np.random.randint(0, 24)
    minute = np.random.randint(0, 60)
    return datetime(2023, 1, 1, hour, minute)  # Fixed date for simplicity

def format_time(dt):
    return dt.strftime('%H:%M')

def final_feature_extraction_with_realistic_times(file_path):
    # Load and clean the dataset
    df = pd.read_csv(file_path)
    df = df.dropna()

    # Remove rows with negative numeric values
    numeric_cols = df.select_dtypes(include=['number']).columns
    df = df[(df[numeric_cols] >= 0).all(axis=1)]

    # Define airport list
    airports = ['JFK', 'LAX', 'ORD', 'ATL', 'DFW', 'DEN', 'SFO', 'SEA', 'MIA', 'PHX']

    # Prepare new feature lists
    num_rows = len(df)
    scheduled_departure = []
    actual_departure = []
    scheduled_arrival = []
    actual_arrival = []
    origins = []
    destinations = []
    flight_statuses = []

    for _ in range(num_rows):
        # Random origin/destination
        origin = np.random.choice(airports)
        destination = np.random.choice([a for a in airports if a != origin])

        # Generate time fields
        sched_dep = generate_random_time()
        dep_delay = timedelta(minutes=np.random.randint(0, 16))  # 0–15 min dep delay
        flight_duration = timedelta(minutes=np.random.randint(60, 121))  # 1–2 hrs
        arr_delay = timedelta(minutes=np.random.randint(0, 31))  # 0–30 min arr delay

        act_dep = sched_dep + dep_delay
        sched_arr = sched_dep + flight_duration
        act_arr = sched_arr + arr_delay

        # Determine flight status
        status = 'delayed' if arr_delay > timedelta(minutes=15) else 'on-time'

        # Store values
        scheduled_departure.append(format_time(sched_dep))
        actual_departure.append(format_time(act_dep))
        scheduled_arrival.append(format_time(sched_arr))
        actual_arrival.append(format_time(act_arr))
        origins.append(origin)
        destinations.append(destination)
        flight_statuses.append(status)

    # Add features to DataFrame
    df['origin'] = origins
    df['destination'] = destinations
    df['scheduled_departure_time'] = scheduled_departure
    df['actual_departure_time'] = actual_departure
    df['scheduled_arrival_time'] = scheduled_arrival
    df['actual_arrival_time'] = actual_arrival
    df['flight_status'] = flight_statuses

    return df

# === Apply Feature Extraction ===
file_path = '/content/drive/MyDrive/flight delay prediction/Airline_Delay_Cleaned.csv'
final_df = final_feature_extraction_with_realistic_times(file_path)

# === Save Updated Dataset ===
output_path = '/content/drive/My Drive/flight delay prediction/final_airline_times_HHMM.csv'
final_df.to_csv(output_path, index=False)

print(f"✅ Dataset saved with realistic times and flight status at: {output_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Dataset saved with realistic times and flight status at: /content/drive/My Drive/flight delay prediction/final_airline_times_HHMM.csv


TRAINING DATASET

In [80]:
# STEP 1: Import your class code
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
import pickle

class FlightDelayPredictor:
    def __init__(self):
        self.model = None
        self.le_origin = LabelEncoder()
        self.le_dest = LabelEncoder()

    def time_to_minutes(self, time_str):
        try:
            h, m = map(int, str(time_str).split(':'))
            return h * 60 + m
        except:
            return np.nan

    def load_and_train(self, path):
        # Corrected Indentation for all lines within the method
        path ='/content/drive/My Drive/flight delay prediction/final_airline_times_HHMM.csv'
        df = pd.read_csv(path)

        df.dropna(subset=[
            'origin', 'destination', 'scheduled_departure_time',
            'scheduled_arrival_time', 'actual_departure_time',
            'actual_arrival_time', 'flight_status'
        ], inplace=True)

        df['sched_dep_min'] = df['scheduled_departure_time'].apply(self.time_to_minutes)
        df['sched_arr_min'] = df['scheduled_arrival_time'].apply(self.time_to_minutes)

        df['dep_delay_min'] = df.apply(
            lambda row: self.time_to_minutes(row['actual_departure_time']) - self.time_to_minutes(row['scheduled_departure_time']),
            axis=1
        )
        df['arr_delay_min'] = df.apply(
            lambda row: self.time_to_minutes(row['actual_arrival_time']) - self.time_to_minutes(row['scheduled_arrival_time']),
            axis=1
        )

        df['origin_enc'] = self.le_origin.fit_transform(df['origin'])
        df['destination_enc'] = self.le_dest.fit_transform(df['destination'])
        df['label'] = df['flight_status'].map({'on-time': 0, 'delayed': 1})

        # IMPORTANT: Only include features available *before* the flight happens to avoid data leakage.
        # 'actual_departure_time', 'actual_arrival_time', 'dep_delay_min', 'arr_delay_min' are NOT available before the flight.
        # Use only scheduled times and airport codes for prediction.
        features = ['origin_enc', 'destination_enc', 'sched_dep_min', 'sched_arr_min']

        # Drop rows where any of the *used* features or the label are missing
        df = df.dropna(subset=features + ['label'])

        X = df[features]
        y = df['label']

        self.model = XGBClassifier(eval_metric='logloss')
        self.model.fit(X, y)

        # ✅ Save trained model
        # The pickle file will be saved in the default Colab environment directory unless a path is specified
        with open('model.pkl', 'wb') as f:
            pickle.dump({
                'model': self.model,
                'le_origin': self.le_origin,
                'le_dest': self.le_dest
            }, f)

        print("✅ Model trained and saved as 'model.pkl'")


    def predict(self, origin, destination, sched_dep, sched_arr): # Removed actual times from prediction input
        if self.model is None:
            raise Exception("Model not trained. Run load_and_train() first.")

        # Prepare input data for prediction using only the features the model was trained on
        try:
            # Ensure inputs are strings before splitting
            input_data = {
                'origin_enc': self.le_origin.transform([str(origin)])[0],
                'destination_enc': self.le_dest.transform([str(destination)])[0],
                'sched_dep_min': self.time_to_minutes(sched_dep),
                'sched_arr_min': self.time_to_minutes(sched_arr),
            }
        except ValueError as e:
             # Provide more specific error message for unknown labels
             if "unseen label" in str(e):
                 raise ValueError(f"Input Error: Origin '{origin}' or destination '{destination}' was not present in the training data.")
             else:
                  raise ValueError(f"Input Error: Could not transform origin '{origin}' or destination '{destination}'. Ensure they were in the training data. Original error: {e}")


        input_df = pd.DataFrame([input_data])

        # Check for NaNs in the prepared input_df (from time conversion)
        if input_df.isnull().any().any():
             raise ValueError("Input Error: Scheduled time format is incorrect (expected HH:MM).")

        # Ensure the input DataFrame has the same column order as the training data features
        # This is crucial for model prediction
        input_df = input_df[['origin_enc', 'destination_enc', 'sched_dep_min', 'sched_arr_min']]


        pred = self.model.predict(input_df)[0]
        return "Delayed" if pred == 1 else "On-Time"


# STEP 3: Train and Save
predictor = FlightDelayPredictor()
# Use the correct path to the generated CSV
predictor.load_and_train('/content/drive/My Drive/flight delay prediction/final_airline_times_HHMM.csv')


✅ Model trained and saved as 'model.pkl'


ACCURACY PREDICTING

In [33]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Load dataset
file_path = '/content/drive/My Drive/flight delay prediction/final_airline_times_HHMM.csv'
df = pd.read_csv(file_path)

# Convert time in HH:MM to total minutes
def time_to_minutes(t):
    h, m = map(int, str(t).split(':'))
    return h * 60 + m

for col in ['scheduled_departure_time', 'scheduled_arrival_time']:
    df[col] = df[col].apply(time_to_minutes)

# Drop columns that cause leakage (these are usually not known at prediction time)
leak_cols = ['actual_departure_time', 'actual_arrival_time', 'arr_del15']  # if they exist
df = df.drop(columns=[col for col in leak_cols if col in df.columns])

# Encode categorical variables (except target)
label_encoders = {}
for col in df.select_dtypes(include='object').columns:
    if col != 'flight_status':
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        label_encoders[col] = le

# Encode target
target_le = LabelEncoder()
df['flight_status'] = target_le.fit_transform(df['flight_status'])  # 'delayed':1, 'on-time':0

# Split data
X = df.drop('flight_status', axis=1)
y = df['flight_status']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("✅ Model Evaluation (Leakage Removed):")
print("Accuracy:", accuracy)
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=target_le.classes_))


✅ Model Evaluation (Leakage Removed):
Accuracy: 0.5514705882352942

Classification Report:
              precision    recall  f1-score   support

     delayed       0.57      0.61      0.59        71
     on-time       0.53      0.49      0.51        65

    accuracy                           0.55       136
   macro avg       0.55      0.55      0.55       136
weighted avg       0.55      0.55      0.55       136



In [34]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Load dataset
file_path = '/content/drive/MyDrive/flight delay prediction/final_airline_times_HHMM.csv'
df = pd.read_csv(file_path)

# Function to convert HH:MM to minutes
def time_to_minutes(t):
    try:
        h, m = map(int, str(t).strip().split(':'))
        return h * 60 + m
    except:
        return np.nan  # Handle invalid/missing time formats

# Apply conversion to time columns
for col in ['scheduled_departure_time', 'scheduled_arrival_time']:
    df[col] = df[col].apply(time_to_minutes)

# Drop rows with any missing values
df.dropna(inplace=True)

# Drop leakage columns (if they exist)
leak_cols = ['actual_departure_time', 'actual_arrival_time', 'arr_del15']
df.drop(columns=[col for col in leak_cols if col in df.columns], inplace=True)

# Encode categorical variables (except target)
label_encoders = {}
for col in df.select_dtypes(include='object').columns:
    if col != 'flight_status':
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        label_encoders[col] = le

# Encode target
target_le = LabelEncoder()
df['flight_status'] = target_le.fit_transform(df['flight_status'])  # delayed=1, on-time=0

# Feature scaling
features = df.drop('flight_status', axis=1)
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Split data (with stratification to preserve label ratio)
X_train, X_test, y_train, y_test = train_test_split(
    features_scaled, df['flight_status'], test_size=0.2, random_state=42, stratify=df['flight_status'])

# Train Random Forest with tuned parameters
model = RandomForestClassifier(
    n_estimators=300,      # Increased trees
    max_depth=15,          # Limit depth to prevent overfitting
    min_samples_split=10,  # More conservative splits
    min_samples_leaf=4,
    random_state=42,
    class_weight='balanced'  # Handle class imbalance
)
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("✅ Model Evaluation (Optimized):")
print("Accuracy:", round(accuracy, 4))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=target_le.classes_))


✅ Model Evaluation (Optimized):
Accuracy: 0.5074

Classification Report:
              precision    recall  f1-score   support

     delayed       0.52      0.56      0.54        70
     on-time       0.49      0.45      0.47        66

    accuracy                           0.51       136
   macro avg       0.51      0.51      0.51       136
weighted avg       0.51      0.51      0.51       136



In [37]:
import pandas as pd
import numpy as np
from datetime import timedelta, datetime
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE

# Utility Functions
def generate_random_time():
    hour = np.random.randint(0, 24)
    minute = np.random.randint(0, 60)
    return datetime(2023, 1, 1, hour, minute)  # Fixed date for simplicity

def format_time(dt):
    return dt.strftime('%H:%M')

# Feature Engineering
def final_feature_extraction_with_realistic_times(file_path):
    df = pd.read_csv(file_path)
    df = df.dropna()

    numeric_cols = df.select_dtypes(include=['number']).columns
    df = df[(df[numeric_cols] >= 0).all(axis=1)]

    airports = ['JFK', 'LAX', 'ORD', 'ATL', 'DFW', 'DEN', 'SFO', 'SEA', 'MIA', 'PHX']
    high_delay_airports = ['JFK', 'ORD', 'ATL']

    scheduled_departure = []
    actual_departure = []
    scheduled_arrival = []
    actual_arrival = []
    origins = []
    destinations = []
    flight_statuses = []

    for _ in range(len(df)):
        origin = np.random.choice(airports)
        destination = np.random.choice([a for a in airports if a != origin])

        sched_dep = generate_random_time()
        sched_dep_hour = sched_dep.hour

        # Increase delay probability for specific airports and times of day
        delay_risk = 0.3
        if origin in high_delay_airports:
            delay_risk += 0.3
        if sched_dep_hour >= 20 or sched_dep_hour <= 6:
            delay_risk += 0.2

        # Decide if delayed based on the calculated delay risk
        is_delayed = np.random.rand() < delay_risk

        # Delay times
        dep_delay = timedelta(minutes=np.random.randint(16, 31) if is_delayed else np.random.randint(0, 6))
        arr_delay = timedelta(minutes=np.random.randint(16, 31) if is_delayed else np.random.randint(0, 6))

        act_dep = sched_dep + dep_delay
        sched_arr = sched_dep + timedelta(minutes=np.random.randint(60, 121))  # 1–2 hrs flight duration
        act_arr = sched_arr + arr_delay

        scheduled_departure.append(format_time(sched_dep))
        actual_departure.append(format_time(act_dep))
        scheduled_arrival.append(format_time(sched_arr))
        actual_arrival.append(format_time(act_arr))
        origins.append(origin)
        destinations.append(destination)
        flight_statuses.append('delayed' if is_delayed else 'on-time')

    df['origin'] = origins
    df['destination'] = destinations
    df['scheduled_departure_time'] = scheduled_departure
    df['actual_departure_time'] = actual_departure
    df['scheduled_arrival_time'] = scheduled_arrival
    df['actual_arrival_time'] = actual_arrival
    df['flight_status'] = flight_statuses

    return df

# Load and prepare the dataset
file_path = '/content/drive/My Drive/flight delay prediction/final_airline_times_HHMM.csv'
final_df = final_feature_extraction_with_realistic_times(file_path)

# Feature extraction and preprocessing
def time_to_minutes(t):
    h, m = map(int, t.split(':'))
    return h * 60 + m

final_df['scheduled_departure_min'] = final_df['scheduled_departure_time'].apply(time_to_minutes)
final_df['scheduled_arrival_min'] = final_df['scheduled_arrival_time'].apply(time_to_minutes)

# Encode categorical variables (origin and destination airports)
le_origin = LabelEncoder()
le_dest = LabelEncoder()

final_df['origin_enc'] = le_origin.fit_transform(final_df['origin'])
final_df['destination_enc'] = le_dest.fit_transform(final_df['destination'])

# Features and Labels
X = final_df[['origin_enc', 'destination_enc', 'scheduled_departure_min', 'scheduled_arrival_min']]
y = final_df['flight_status'].map({'on-time': 0, 'delayed': 1})

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Handle Class Imbalance using SMOTE (Synthetic Minority Over-sampling Technique)
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Create and train the Random Forest model with hyperparameters tuning
rf_model = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42, class_weight='balanced')

# Fit the model
rf_model.fit(X_train_smote, y_train_smote)

# Predict on the test set
y_pred = rf_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"✅ Accuracy: {accuracy:.4f}\n")

print("📊 Classification Report:")
print(classification_report(y_test, y_pred, target_names=['on-time', 'delayed']))



✅ Accuracy: 0.5956

📊 Classification Report:
              precision    recall  f1-score   support

     on-time       0.61      0.61      0.61        71
     delayed       0.58      0.58      0.58        65

    accuracy                           0.60       136
   macro avg       0.60      0.60      0.60       136
weighted avg       0.60      0.60      0.60       136



In [40]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE

# Convert time string HH:MM to minutes
def time_to_minutes(t):
    h, m = map(int, t.split(':'))
    return h * 60 + m

def extract_hour(t):
    return int(t.split(":")[0])

def is_night_flight(hour):
    return 1 if (hour <= 6 or hour >= 22) else 0

# Load your enhanced dataset
file_path = '/content/drive/My Drive/flight delay prediction/final_airline_times_HHMM.csv'
df = pd.read_csv(file_path)

# Clean missing and invalid entries
df = df.dropna()

# Label encoding for categorical
le_origin = LabelEncoder()
le_dest = LabelEncoder()

df['origin_enc'] = le_origin.fit_transform(df['origin'])
df['destination_enc'] = le_dest.fit_transform(df['destination'])

# Time features
df['scheduled_departure_min'] = df['scheduled_departure_time'].apply(time_to_minutes)
df['scheduled_arrival_min'] = df['scheduled_arrival_time'].apply(time_to_minutes)
df['departure_hour'] = df['scheduled_departure_time'].apply(extract_hour)
df['night_flight'] = df['departure_hour'].apply(is_night_flight)

# Duration (proxy)
df['flight_duration'] = df['scheduled_arrival_min'] - df['scheduled_departure_min']
df['flight_status'] = df['flight_status'].map({'on-time': 0, 'delayed': 1})

# Define features and labels
features = [
    'origin_enc',
    'destination_enc',
    'scheduled_departure_min',
    'scheduled_arrival_min',
    'departure_hour',
    'night_flight',
    'flight_duration'
]
X = df[features]
y = df['flight_status']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Handle imbalance with SMOTE
smote = SMOTE(random_state=42)
X_train_sm, y_train_sm = smote.fit_resample(X_train, y_train)

# Create optimized model
model = RandomForestClassifier(
    n_estimators=300,
    max_depth=12,
    min_samples_split=5,
    min_samples_leaf=2,
    class_weight='balanced',
    random_state=42
)
model.fit(X_train_sm, y_train_sm)

# Evaluate
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print(f"✅ Boosted Accuracy: {acc:.4f}")
print("📊 Classification Report:")
print(classification_report(y_test, y_pred, target_names=['on-time', 'delayed']))


✅ Boosted Accuracy: 0.5221
📊 Classification Report:
              precision    recall  f1-score   support

     on-time       0.50      0.58      0.54        65
     delayed       0.55      0.46      0.50        71

    accuracy                           0.52       136
   macro avg       0.53      0.52      0.52       136
weighted avg       0.53      0.52      0.52       136



In [43]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier

# Load the cleaned dataset
# Note: This script will overwrite the time and airport data in this CSV with synthetic data.
file_path = '/content/drive/My Drive/flight delay prediction/final_airline_times_HHMM.csv'
df = pd.read_csv(file_path)

# Step 1: Generate synthetic time and airport features ensuring both classes exist
airports = ['JFK', 'LAX', 'ORD', 'ATL', 'DFW', 'DEN', 'SFO', 'SEA', 'MIA', 'PHX']
origins, destinations = [], []
scheduled_departure, actual_departure = [], []
scheduled_arrival, actual_arrival = [], []
statuses, dep_delay_mins_list, arr_delay_mins_list = [], [], [] # Changed variable names

num_rows = len(df)
# Define the target proportion of delayed flights (e.g., 20%)
delayed_proportion = 0.2
num_delayed = int(num_rows * delayed_proportion)
num_ontime = num_rows - num_delayed

# Create a list of statuses to assign, ensuring the desired proportion
all_statuses = ['Delayed'] * num_delayed + ['On-Time'] * num_ontime
random.shuffle(all_statuses) # Randomize the order

for status in all_statuses:
    origin = random.choice(airports)
    destination = random.choice([a for a in airports if a != origin])

    sched_dep = datetime.strptime(f"{random.randint(5, 20):02d}:{random.choice([0,15,30,45]):02d}", '%H:%M')
    # Random flight duration between 1 and 5 hours
    flight_duration = timedelta(hours=random.randint(1, 5), minutes=random.randint(0, 59)) # Add random minutes
    sched_arr = sched_dep + flight_duration

    if status == 'Delayed':
        # Ensure actual_arrival - actual_departure > 15 minutes
        # Actual Arrival - Actual Departure = (Scheduled Arrival + Arr Delay) - (Scheduled Departure + Dep Delay)
        # = (Scheduled Arrival - Scheduled Departure) + (Arr Delay - Dep Delay)
        # = Flight Duration + (Arr Delay - Dep Delay)
        # We need Flight Duration + (Arr Delay - Dep Delay) > timedelta(minutes=15)
        # This condition is incorrect based on the original logic (actual_dep - actual_arr > 15 min).
        # Let's stick to the original status condition: actual_dep - actual_arr > 15 min
        # This means (sched_dep + dep_delay) - (sched_arr + arr_delay) > timedelta(minutes=15)
        # which is dep_delay - arr_delay - flight_duration > timedelta(minutes=15)
        # This still seems counter-intuitive for a "delayed" status based on actual times.
        # A more standard definition of arrival delay is Actual Arrival - Scheduled Arrival.
        # Let's assume the intended "Delayed" status meant Actual Arrival is more than 15 minutes after Scheduled Arrival.
        # status = 'Delayed' if (actual_arr - sched_arr) > timedelta(minutes=15) else 'On-Time'
        # If that's the case, let's generate delays accordingly.

        # Let's revise the delay generation to align with a common definition of delay:
        # A flight is delayed if Actual Arrival > Scheduled Arrival + 15 minutes.
        # This means arr_delay_min > 15.
        arr_delay_min = random.randint(16, 60) # Ensure > 15 min arrival delay
        dep_delay_min = random.randint(0, 60)   # Departure delay can be anything

    else: # status == 'On-Time'
        # Ensure Actual Arrival <= Scheduled Arrival + 15 minutes.
        # This means arr_delay_min <= 15.
        arr_delay_min = random.randint(0, 15)  # Ensure <= 15 min arrival delay
        dep_delay_min = random.randint(0, 60)  # Departure delay can be anything

    dep_delay = timedelta(minutes=dep_delay_min)
    arr_delay = timedelta(minutes=arr_delay_min)

    actual_dep = sched_dep + dep_delay
    actual_arr = sched_arr + arr_delay

    # Store values
    origins.append(origin)
    destinations.append(destination)
    scheduled_departure.append(sched_dep.strftime('%H:%M'))
    actual_departure.append(actual_dep.strftime('%H:%M'))
    scheduled_arrival.append(sched_arr.strftime('%H:%M'))
    actual_arrival.append(actual_arr.strftime('%H:%M'))
    statuses.append(status)
    dep_delay_mins_list.append(dep_delay_min)
    arr_delay_mins_list.append(arr_delay_min)


# Add generated features to dataframe, replacing existing ones
df['Origin'] = origins
df['Destination'] = destinations
df['Scheduled_Departure'] = scheduled_departure
df['Actual_Departure'] = actual_departure # Keep actual times for reference, but won't use for prediction
df['Scheduled_Arrival'] = scheduled_arrival
df['Actual_Arrival'] = actual_arrival   # Keep actual times for reference, but won't use for prediction
df['Flight_Status'] = statuses
# Keep these generated delay columns as potential features
df['Dep_Delay_Min'] = dep_delay_mins_list
df['Arr_Delay_Min'] = arr_delay_mins_list # This is leakage if used to predict Flight_Status defined by Arr_Delay_Min


# Feature Engineering
# Convert time strings HH:MM to total minutes since midnight
def time_to_minutes(t):
    h, m = map(int, t.split(':'))
    return h * 60 + m

df['Scheduled_Departure_Min'] = df['Scheduled_Departure'].apply(time_to_minutes)
df['Scheduled_Arrival_Min'] = df['Scheduled_Arrival'].apply(time_to_minutes)

# Encode airport codes
le_origin = LabelEncoder()
le_dest = LabelEncoder()
df['Origin_enc'] = le_origin.fit_transform(df['Origin'])
df['Destination_enc'] = le_dest.fit_transform(df['Destination'])

# Label encoding for target
# Ensure the mapping matches the string statuses generated ('On-Time', 'Delayed')
df['Flight_Status_Label'] = df['Flight_Status'].map({'On-Time': 0, 'Delayed': 1})

# Define features and target label
# IMPORTANT: Do NOT include 'Actual_Departure', 'Actual_Arrival', or 'Arr_Delay_Min'
# if your goal is to predict flight status *before* the flight happens.
# 'Dep_Delay_Min' might also be leakage depending on the prediction point,
# but often departure delay influences arrival delay. Let's include it for now,
# but be aware this might not be available at the time of prediction.
features = ['Origin_enc', 'Destination_enc', 'Scheduled_Departure_Min',
            'Scheduled_Arrival_Min', 'Dep_Delay_Min'] # Removed Arr_Delay_Min

X = df[features]
y = df['Flight_Status_Label']

# Train-test split
# Use stratification to ensure the split preserves the ratio of 'On-Time' vs 'Delayed'
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


# Train XGBoost Classifier
# Add scale_pos_weight to handle potential class imbalance even after controlled generation
# It's 1 if balanced, > 1 if minority class (1: Delayed) is underrepresented.
# You can calculate it as count(negative_class) / count(positive_class)
# For binary classification, it's usually count(0) / count(1).
# Let's calculate it from the training data
scale_pos_weight_value = (y_train == 0).sum() / (y_train == 1).sum()

model = XGBClassifier(eval_metric='logloss', random_state=42)
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("✅ Accuracy:", round(accuracy * 100, 2), "%")
print("\n📊 Classification Report:\n", classification_report(y_test, y_pred, target_names=["On-Time", "Delayed"]))

✅ Accuracy: 75.0 %

📊 Classification Report:
               precision    recall  f1-score   support

     On-Time       0.82      0.89      0.85       109
     Delayed       0.29      0.19      0.23        27

    accuracy                           0.75       136
   macro avg       0.55      0.54      0.54       136
weighted avg       0.71      0.75      0.73       136



PREDICTIOM MODULE

In [52]:
!pip install xgboost


In [61]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

class FlightDelayPredictor:
    def __init__(self):
        self.model = None
        self.le_origin = LabelEncoder()
        self.le_dest = LabelEncoder()

    def time_to_minutes(self, time_str):
        try:
            h, m = map(int, str(time_str).split(':')) # Ensure time_str is string
            return h * 60 + m
        except:
            return np.nan

    def load_and_train(self, path):
        df = pd.read_csv(path)

        # Drop rows with missing critical time info
        df.dropna(subset=[
            'origin', 'destination', 'scheduled_departure_time',
            'scheduled_arrival_time', 'actual_departure_time',
            'actual_arrival_time', 'flight_status'
        ], inplace=True)

        # Feature Engineering
        df['sched_dep_min'] = df['scheduled_departure_time'].apply(self.time_to_minutes)
        df['sched_arr_min'] = df['scheduled_arrival_time'].apply(self.time_to_minutes)
        # Calculate delays - ensure time_to_minutes handles potential NaNs or errors gracefully
        df['dep_delay_min'] = df.apply(
            lambda row: self.time_to_minutes(row['actual_departure_time']) - self.time_to_minutes(row['scheduled_departure_time']),
            axis=1
        )
        df['arr_delay_min'] = df.apply(
            lambda row: self.time_to_minutes(row['actual_arrival_time']) - self.time_to_minutes(row['scheduled_arrival_time']),
            axis=1
        )

        df['origin_enc'] = self.le_origin.fit_transform(df['origin'])
        df['destination_enc'] = self.le_dest.fit_transform(df['destination'])
        df['label'] = df['flight_status'].map({'on-time': 0, 'delayed': 1})

        # Features used for training the model
        # IMPORTANT: Only include features available *before* the flight happens to avoid data leakage.
        # 'actual_departure_time', 'actual_arrival_time', 'dep_delay_min', 'arr_delay_min' are NOT available before the flight.
        # Use only scheduled times and airport codes for prediction.
        features = ['origin_enc', 'destination_enc', 'sched_dep_min', 'sched_arr_min']

        # Drop rows where any of the *used* features or the label are missing
        df = df.dropna(subset=features + ['label'])


        X = df[features]
        y = df['label']

        # Corrected Indentation
        self.model = XGBClassifier(eval_metric='logloss')

        self.model.fit(X, y)

    def predict(self, origin, destination, sched_dep, sched_arr): # Removed actual times from prediction input
        if self.model is None:
            raise Exception("Model not trained. Run load_and_train() first.")

        # Prepare input data for prediction using only the features the model was trained on
        try:
            input_data = {
                'origin_enc': self.le_origin.transform([origin])[0],
                'destination_enc': self.le_dest.transform([destination])[0],
                'sched_dep_min': self.time_to_minutes(sched_dep),
                'sched_arr_min': self.time_to_minutes(sched_arr),
            }
        except ValueError as e:
             raise ValueError(f"Input Error: Could not transform origin '{origin}' or destination '{destination}'. Ensure they were in the training data. Original error: {e}")

        input_df = pd.DataFrame([input_data])

        # Check for NaNs in the prepared input_df
        if input_df.isnull().any().any():
             raise ValueError("Input Error: Scheduled time format is incorrect (expected HH:MM).")

        pred = self.model.predict(input_df)[0]
        return "Delayed" if pred == 1 else "On-Time"

In [63]:
predictor = FlightDelayPredictor()

# Train the model using your uploaded dataset
predictor.load_and_train('/content/drive/My Drive/flight delay prediction/final_airline_times_HHMM.csv')

# Make a sample prediction
result = predictor.predict(
    origin='SFO',
    destination='DFW',
    sched_dep='18:44',
    sched_arr='20:37'
)

print("🛫 Prediction Result:", result)

🛫 Prediction Result: Delayed
